In [ ]:
import numpy as np

In [ ]:
a = np.random.random_sample((3,3))

In [ ]:
a


array([[0.02497302, 0.22954173, 0.58889283],
       [0.57688675, 0.19181968, 0.1610521 ],
       [0.90875011, 0.06123859, 0.05602084]])

In [ ]:
row_maxes = a.max(axis=1).reshape(-1, 1)

In [ ]:
a[:] = np.where(a == row_maxes, 1, 0)

In [ ]:
a

array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [30]:
import pandas as pd
y_true = pd.read_csv('/content/file1 (1).csv')

In [31]:
y_true = y_true.iloc[:,1::]

In [32]:
y_true

,file,label
0,/content/test/covid 19/HUST-Patient1396-0299.png,covid
1,/content/test/covid 19/NCP_952_2495_0153.png,covid
2,/content/test/covid 19/HUST-Patient1330-0238.png,covid
3,/content/test/covid 19/137covid_patient89_SR_3...,covid
4,/content/test/covid 19/HUST-Patient1386-0214.png,covid
...,...,...
9743,/content/test/pneumonia/CP_2454_2936_0048.png,pneumonia
9744,/content/test/pneumonia/CP_1607_4295_0006.png,pneumonia
9745,/content/test/pneumonia/CP_672_3034_0036.png,pneumonia
9746,/content/test/pneumonia/CP_729_3091_0015.png,pneumonia


In [13]:
pip install patool

     |████████████████████████████████| 77 kB 4.6 MB/s 


In [16]:
import patoolib
patoolib.extract_archive("/content/drive/MyDrive/test.rar", outdir="/content/")

patool: Extracting /content/drive/MyDrive/test.rar ...
patool: running /usr/bin/unrar x -- /content/drive/MyDrive/test.rar
patool:     with cwd='/content/'
patool: ... /content/drive/MyDrive/test.rar extracted to `/content/'.


'/content/'

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# import tensorflow as tf
# import glob
# from keras.layers import Dense, Activation, Flatten , GlobalAveragePooling2D, Dropout
# from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau


In [18]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, Callback

In [19]:
def new_layer(model):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation="relu")(x)
    predictions = Dense(3, activation="softmax")(x)
    return predictions

In [20]:
!pip install efficientnet

     |████████████████████████████████| 50 kB 4.9 MB/s 


In [21]:
import efficientnet.tfkeras as efn
model1 =efn.EfficientNetB4(weights ='noisy-student', include_top=False, input_shape = (348,348,3))
for layer in model1.layers:
  layer.trainable = False
model2 = Model(inputs=model1.input, outputs=new_layer(model1))
# x = Flatten(model1.output)
# prediction = Dense(3, activation='softmax')(x)
# model = Model(inputs = model1.input, outputs = prediction)


71688192/71678424 [==============================] - 1s 0us/step


In [22]:
model2.compile(loss = "categorical_crossentropy", optimizer= tf.keras.optimizers.Adam(), metrics=["accuracy"])

In [23]:
checkpoint_path = "/content/drive/MyDrive/4"

ckpt = tf.train.Checkpoint(model2)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [24]:
test_datagen = ImageDataGenerator(horizontal_flip=True,
    vertical_flip=True,
    rotation_range=360,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=.2,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.2,
    brightness_range=[0.2, 2.0])

In [37]:
test_data = test_datagen.flow_from_dataframe(y_true,
                                              batch_size= 16,
                                              target_size=(348,348),
                                              x_col="file",
                                              y_col = 'label',
                                             shuffle = False,
                                              class_mode="raw")

Found 9748 validated image filenames.


In [38]:
y_pred = []

In [39]:
y_pred.append(model2.predict(test_data,steps=test_data.n/16,verbose=1))

609/609 [==============================] - 347s 570ms/step


In [44]:
y_pred

ValueError: ignored